In [ ]:
# List of pipeline steps

**FeatureGenerationPipeline:**
- CustomAttributeAdder(feature_list=['feature1', 'feature2', 'feature3', etc.]):
- RandomAttributeAdder(stratgey=['multiply', 'add', 'substract', 'divide']):

**PreprocessingPipeline:**
- Imputer???
- RandomAttributeTransformer(strategy=['sqaure', 'log', 'sqrt'])
- Balancing(stratgey=['up', 'down', 'other', 'none'])
- Scaling(strategy=['Standard', 'MinMax', 'None'])

**ModelSelectionPipeline:**
- ModelSelector
- CrossValidation


In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt

from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.model_selection import cross_validate, StratifiedKFold, GridSearchCV
from sklearn.metrics import make_scorer

# classifiers
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier

# use sklearn pipeline
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.impute import SimpleImputer

# custom imports
from funcs import plot_cv_confidence_vs_profit, score_dmc_profit,dmc_profit,cv_preds_and_confusion_matrix
from customClassifiers import CustomModelWithThreshold

# 1. Load Data

In [2]:
df_train = pd.read_csv('train.csv', sep='|')
df_test = pd.read_csv('test.csv', sep='|')

df_train.head()
df_train.tail()
df_train.sample(3)

,trustLevel,totalScanTimeInSeconds,grandTotal,lineItemVoids,scansWithoutRegistration,quantityModifications,scannedLineItemsPerSecond,valuePerSecond,lineItemVoidsPerPosition,fraud
981,6,1316,14.22,5,10,3,0.015957,0.010805,0.238095,0
364,5,1487,51.83,3,7,1,0.001345,0.034855,1.500000,0
528,4,1796,88.53,0,9,0,0.001114,0.049293,0.000000,0


In [3]:
# split label and predictors
X_train, y_train = df_train.drop(columns=['fraud',]), df_train['fraud']
X_train.head()

,trustLevel,totalScanTimeInSeconds,grandTotal,lineItemVoids,scansWithoutRegistration,quantityModifications,scannedLineItemsPerSecond,valuePerSecond,lineItemVoidsPerPosition
0,5,1054,54.70,7,0,3,0.027514,0.051898,0.241379
1,3,108,27.36,5,2,4,0.129630,0.253333,0.357143
2,3,1516,62.16,3,10,5,0.008575,0.041003,0.230769
3,6,1791,92.31,8,4,4,0.016192,0.051541,0.275862
4,5,430,81.53,3,7,2,0.062791,0.189605,0.111111


# 2. Pipeline

### 2.1 FeatureGeneration Pipeline

In [4]:
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler

In [5]:
# scaling class
class Scaling(TransformerMixin):
    
    _strategy = "Standard"
    _scaler = None
    
    def __init__(self, strategy):
        self._strategy = strategy
    
    def fit(self, X, y=None):
        if self._strategy == "Standard":
            self._scaler = StandardScaler()
            return self._scaler.fit(X)
        elif self._strategy == "MinMax":
            self._scaler = MinMaxScaler()
            return self._scaler.fit(X)
        elif self._strategy == "None":
            return self
    
    def transform(self, X):
        
        if self._strategy == "None":
            return X
        else:
            return self._scaler.transform(X)
        

In [6]:
# custom attribute adder
class CustomAttributeAdder(TransformerMixin):
    
    _featureList = ['scannedLineItemsTotal', 'valuePerLineItem', 'quantityModificationsPerLineItem']
    
    def __init__(self, feature_list = ['scannedLineItemsTotal',
                                       'valuePerLineItem',
                                       'quantityModificationsPerLineItem']):
        self.__featurelist = feature_list
        
    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        
        X['scannedLineItemsTotal'] = X['scannedLineItemsPerSecond'] * X['totalScanTimeInSeconds']
            
        if "valuePerLineItem" in self._featureList:
            X['valuePerLineItem'] = X['grandTotal'] / X['scannedLineItemsTotal']
            
        if "quantityModificationsPerLineItem" in self._featureList:
            X['quantityModificationsPerLineItem'] = X['quantityModifications'] / X['scannedLineItemsTotal']
            
        return X

In [7]:
# custom attribute adder
class RandomAttributeAdder(TransformerMixin):
    
    """This class is still empty and needs to be filled!"""
    
    def __init__(self,):
        pass
    def fit(self, X, y=None):
        return self
    
    def transform(self, X):    
        return X

In [8]:
# featureGeneration pipeline 
# all transform methods should return a pd.DataFrame

featureGeneration_pipeline = Pipeline([
    ("attribs_adder", CustomAttributeAdder()),    # returns pd.dataframe
    ("RandomAttributeAdder", RandomAttributeAdder())
    ])

### 2.2 Preprocessing Pipeline

In [9]:
class Transformer(TransformerMixin):
    
    """This class is still empty and needs to be filled!"""
    
    def __init__(self,):
        pass
    def fit(self, X, y=None):
        return self
    
    def transform(self, X):    
        return X

In [10]:
class Balancing(TransformerMixin):
    
    """This class is still empty and needs to be filled!"""
    
    def __init__(self,):
        pass
    def fit(self, X, y=None):
        return self
    
    def transform(self, X):    
        return X

In [11]:
# scaling class
class Scaling(TransformerMixin):
    
    _strategy = "Standard"
    _scaler = None
    
    def __init__(self, strategy):
        self._strategy = strategy
    
    def fit(self, X, y=None):
        if self._strategy == "Standard":
            self._scaler = StandardScaler()
            return self._scaler.fit(X)
        elif self._strategy == "MinMax":
            self._scaler = MinMaxScaler()
            return self._scaler.fit(X)
        elif self._strategy == "None":
            return self
    
    def transform(self, X):
        
        if self._strategy == "None":
            return X
        else:
            return self._scaler.transform(X)
      

In [12]:
# In this pipeline, we are working with np.array

preprocessing_pipeline = Pipeline([
    ("imputer", SimpleImputer(strategy="mean")),   # This step is actually not needed
    ("transformer", Transformer()),                # This class is still void
    ("balancing", Balancing()),                    # This class is still void
    ("scaler", Scaling(strategy='Standard')),
])

### 2.3 Model Training Pipeline

In [13]:
from sklearn.base import BaseEstimator
from sklearn.linear_model import SGDClassifier

class ClfSwitcher(BaseEstimator):

    def __init__(
        self, 
        estimator = SGDClassifier()
    ):
        """
        A Custom BaseEstimator that can switch between classifiers.
        :param estimator: sklearn object - The classifier
        """ 

        self.estimator = estimator


    def fit(self, X, y=None, **kwargs):
        self.estimator.fit(X, y)
        return self


    def predict(self, X, y=None):
        return self.estimator.predict(X)


    def predict_proba(self, X):
        return self.estimator.predict_proba(X)


    def score(self, X, y):
        return self.estimator.score(X, y)

In [14]:
model_training_pipeline = Pipeline([
    ('feature_generation', featureGeneration_pipeline),
    ('preprocessing', preprocessing_pipeline),
    ('classifier', ClfSwitcher())
])

In [15]:
parameters = [
    {
        'classifier__estimator': [SGDClassifier()],
        'classifier__estimator__penalty': ('l2', 'elasticnet', 'l1'),
        'classifier__estimator__max_iter': [50, 80],
        'classifier__estimator__tol': [1e-4],
        'classifier__estimator__loss': ['hinge', 'log', 'modified_huber']
    },
    {
        'classifier__estimator': [XGBClassifier()],
        'classifier__estimator__n_estimators': [50, 100, 150],
        'classifier__estimator__reg_alpha': [0, 0.05, 0.1]
    },
    {
        'classifier__estimator': [RandomForestClassifier()],
        'classifier__estimator__min_samples_split': [2, 4, 6],
        'classifier__estimator__criterion': ['gini', 'entropy']
    }
]

# 3. Full Pipeline Usage Example

In [16]:
cv = StratifiedKFold(n_splits=10, random_state=42)

def profit_scorer(y, y_pred):
    profit_matrix = {(0,0): 0, (0,1): -5, (1,0): -25, (1,1): 5}
    return sum(profit_matrix[(pred, actual)] for pred, actual in zip(y_pred, y))
profit_scoring = make_scorer(profit_scorer, greater_is_better=True)

In [17]:
gscv = GridSearchCV(model_training_pipeline, parameters, cv=10, n_jobs=-1, scoring=profit_scoring, verbose=3)
gscv.fit(X_train, y_train)

Fitting 10 folds for each of 33 candidates, totalling 330 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed:    4.0s
[Parallel(n_jobs=-1)]: Done 112 tasks      | elapsed:    8.8s
[Parallel(n_jobs=-1)]: Done 272 tasks      | elapsed:   18.7s
[Parallel(n_jobs=-1)]: Done 330 out of 330 | elapsed:   21.5s finished
/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_search.py:841: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


GridSearchCV(cv=10, error_score='raise-deprecating',
       estimator=Pipeline(memory=None,
     steps=[('feature_generation', Pipeline(memory=None,
     steps=[('attribs_adder', <__main__.CustomAttributeAdder object at 0x1a1db87b70>), ('RandomAttributeAdder', <__main__.RandomAttributeAdder object at 0x1a1db87c88>)])), ('preprocessing', Pipeline(memory=None,
     steps=[('imputer', SimpleImpute..._state=None, shuffle=True, tol=None,
       validation_fraction=0.1, verbose=0, warm_start=False)))]),
       fit_params=None, iid='warn', n_jobs=-1,
       param_grid=[{'classifier__estimator': [SGDClassifier(alpha=0.0001, average=False, class_weight=None,
       early_stopping=False, epsilon=0.1, eta0=0.0, fit_intercept=True,
       l1_ratio=0.15, learning_rate='optimal', loss='modified_huber',
       max_iter=50, n_iter=None, n_iter_no_change=5, n_jo..._estimator__min_samples_split': [2, 4, 6], 'classifier__estimator__criterion': ['gini', 'entropy']}],
       pre_dispatch='2*n_jobs', refit=

In [18]:
gscv.best_score_

29.989356040447046

In [19]:
gscv.best_params_

{'classifier__estimator': SGDClassifier(alpha=0.0001, average=False, class_weight=None,
        early_stopping=False, epsilon=0.1, eta0=0.0, fit_intercept=True,
        l1_ratio=0.15, learning_rate='optimal', loss='modified_huber',
        max_iter=50, n_iter=None, n_iter_no_change=5, n_jobs=None,
        penalty='elasticnet', power_t=0.5, random_state=None, shuffle=True,
        tol=0.0001, validation_fraction=0.1, verbose=0, warm_start=False),
 'classifier__estimator__loss': 'modified_huber',
 'classifier__estimator__max_iter': 50,
 'classifier__estimator__penalty': 'elasticnet',
 'classifier__estimator__tol': 0.0001}

In [20]:
gscv.best_estimator_.named_steps

{'feature_generation': Pipeline(memory=None,
      steps=[('attribs_adder', <__main__.CustomAttributeAdder object at 0x1a1f737898>), ('RandomAttributeAdder', <__main__.RandomAttributeAdder object at 0x1a1f737128>)]),
 'preprocessing': Pipeline(memory=None,
      steps=[('imputer', SimpleImputer(copy=True, fill_value=None, missing_values=nan, strategy='mean',
        verbose=0)), ('transformer', <__main__.Transformer object at 0x109f2f518>), ('balancing', <__main__.Balancing object at 0x1a1f712080>), ('scaler', <__main__.Scaling object at 0x1a1f721ba8>)]),
 'classifier': ClfSwitcher(estimator=SGDClassifier(alpha=0.0001, average=False, class_weight=None,
        early_stopping=False, epsilon=0.1, eta0=0.0, fit_intercept=True,
        l1_ratio=0.15, learning_rate='optimal', loss='modified_huber',
        max_iter=50, n_iter=None, n_iter_no_change=5, n_jobs=None,
        penalty='elasticnet', power_t=0.5, random_state=None, shuffle=True,
        tol=0.0001, validation_fraction=0.1, verbose